# Investigando algo genético para planear torneos *Double Round Robin*

En torneos Double Round Robin todos los equipos se enfrentan 2 veces, una vez en casa y otra de visita.

Como conocemos todas las partidas de antemano, la planeación de un torneo se reduce a ordenar las partidas de una forma válida/óptima. Para que una asignacion sea valida cada equipo debe jugar exactamente una vez por jornada.

Si hay n equipos hay
- n(n-1) juegos en el torneo
- n/2 juegos por jornada
- y 2(n-1) jornadas.

Representamos una partida con la pareja X,Y, que indica que juega X vs. Y en casa de X.

Codigo en ingles. Jornada->week.

In [3]:
import numpy as np
import random,time,json
from tqdm import tqdm
import os

In [4]:
# Given the number of teams, returns the list of games that must be played
def games(nTeams):
    games=[]
    for i in range(1,nTeams+1):
        for j in range(1,nTeams+1):
            if i==j:continue
            games.append((i,j))
    assert len(games)==nTeams*(nTeams-1)
    return games

In [5]:
# Returns a random assignment/tournament by shuffling the list of games
def randomAssignment(nTeams):
    p=games(nTeams)
    random.shuffle(p)
    return p

In [6]:
# Returns a random assignment of teams to cities
def randomCityTeams(nTeams,nCities):
    cities={i:[] for i in range(1,nCities+1)} # city:[team1,team2]
    randTeams=list(range(1,nTeams+1))
    random.shuffle(randTeams)
    while len(randTeams)>0:
        city=random.choice(range(1,nCities+1))
        cities[city].append(randTeams.pop())
    return cities

In [7]:
# Given a cityTeams dict (city:[list of teams]), returns a dict (team:city)
def teamToCity(cityTeams):
    out={}
    for city,teams in cityTeams.items():
        for team in teams:
            out[team]=city
    return out

In [8]:
def randomCityDistances(nCities,low=0,high=100):
    r=np.random.randint(low,high,size=(nCities,nCities))
    return (r+r.T)/2

In [9]:
randomCityDistances(4)

array([[80. , 64.5, 27. , 43.5],
       [64.5, 20. , 71.5, 76. ],
       [27. , 71.5, 12. , 24. ],
       [43.5, 76. , 24. , 12. ]])

In [10]:
randomCityTeams(4,3)

{1: [1, 3], 2: [4], 3: [2]}

In [11]:
randomAssignment(4)

[(4, 3),
 (2, 1),
 (1, 4),
 (4, 2),
 (4, 1),
 (2, 3),
 (3, 1),
 (3, 4),
 (1, 3),
 (3, 2),
 (2, 4),
 (1, 2)]

In [12]:
def lastWeek(assignment,nTeams):
    week=[]
    for i in range(0,len(assignment),nTeams//2):
        week=assignment[i:i+nTeams//2]
    return week

In [127]:
def approx(nTeams):
    current=[]
    gs=games(nTeams)
    remaining=gs[:]
    random.shuffle(remaining)
    week=[]
    while len(current)!=nTeams*(nTeams-1):
        available=[]
        for game in remaining:
            if game[0] not in week and game[1] not in week:
                available.append(game)
        game=random.choice(available) if available else remaining.pop()
        current.append(game)
        week.append(game)
        week=week[-nTeams//2:]
        remaining.remove(game)
    assert len(set(current))==nTeams*(nTeams-1)
    return current   

In [128]:
approx(4)

[(2, 1),
 (1, 2),
 (3, 2),
 (4, 3),
 (3, 4),
 (3, 1),
 (1, 3),
 (1, 4),
 (2, 4),
 (2, 3),
 (4, 1),
 (4, 2)]

In [13]:
def approxAssignment(nTeams):
    current=[]
    gs=games(nTeams)
    remaining=gs[:]
    while len(current)!=nTeams*(nTeams-1):
        available=[]
        inLastWeek=teamsInGames(lastWeek(current,nTeams))
        for game in remaining:
            if game[0] not in inLastWeek and game[1] not in inLastWeek:
                available.append(game)
        game=random.choice(available) if available else random.choice(remaining)
        current.append(game)
        remaining.remove(game)
    assert len(set(current))==nTeams*(nTeams-1)
    return current

In [14]:
# Used in conflicts(). Given a list of len 2 tuples, returns a list with all elements.
def teamsInGames(week):
    out=[]
    for game in week:
        out.append(game[0])
        out.append(game[1])
    return out

In [130]:
# Returns the conflicts in an assignment/tournament.
# Marks as 1 games in which a participating team plays more than once in that week.
def conflicts(assignment,nTeams):
    conflicts=[]
    n=nTeams
    for i in range(0,len(assignment),n//2):
        week=assignment[i:i+n//2]
        print(week)
        teamsInWeek=teamsInGames(week)
        for game in week:
            conflicts.append(0)
            for team in game:
                if teamsInWeek.count(team)!=1:
                    conflicts[-1]=1
    return conflicts

In [ ]:
[[4,3],[3,4],[1,3],[1,2],[2,4],[1,4],[4,2],[2,3],[3,2],[4,1],[2,1],[3,1]]

In [133]:
for a,b in p:
    print('['+str(a)+','+str(b)+'],',end='')
    

[4,3],[3,4],[1,3],[1,2],[2,4],[1,4],[4,2],[2,3],[3,2],[4,1],[2,1],[3,1],

In [16]:
p=randomAssignment(4)
p

[(4, 3),
 (3, 4),
 (1, 3),
 (1, 2),
 (2, 4),
 (1, 4),
 (4, 2),
 (2, 3),
 (3, 2),
 (4, 1),
 (2, 1),
 (3, 1)]

In [131]:
conflicts(p,4)

[(4, 3), (3, 4)]
[(1, 3), (1, 2)]
[(2, 4), (1, 4)]
[(4, 2), (2, 3)]
[(3, 2), (4, 1)]
[(2, 1), (3, 1)]


[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]

In [18]:
# Returns the number of conflicts in an assignment. 
def nConflicts(assignment,nTeams):
    return sum(conflicts(assignment,nTeams))

## Min conflits (base line)

Para comparar el algo genetico con algo, intente implementar Min Conflicts (ver busqueda local en el lib) con swaps.

In [14]:
# Given the index of a conflict, return the game swap that results least nConflicts.
def minSwap(assignment,conflict,nTeams):
    bestN,bestOthers=None,[]
    for i in range(len(assignment)):
        assignment[conflict],assignment[i]=assignment[i],assignment[conflict]
        n=nConflicts(assignment,nTeams)
        if not bestN or n==bestN:
            bestN=n
            bestOthers.append(i)
        elif bestN and n<bestN:
            bestN=n
            bestOthers=[i]       
        assignment[conflict],assignment[i]=assignment[i],assignment[conflict]
    return random.choice(bestOthers)

In [15]:
# Returns the index of a random conflict in assignment.
def randomConflict(assignment,nTeams):
    indices=np.argwhere(conflicts(assignment,nTeams))
    return indices[np.random.randint(0,indices.shape[0])][0]

In [16]:
# Min Conflicts implementation.
# Returns final state of current and wether it was solved (nConflicts==0)
def minConflicts(nTeams,maxSteps,iniApprox=True,debug=False,autoEndK=100):
    current=approxAssignment(nTeams) if iniApprox else randomAssignment(nTeams)
    n=nConflicts(current,nTeams)
    lastK=[]
    for i in range(maxSteps):
        n=nConflicts(current,nTeams)
        lastK.append(n)
        lastK=lastK[-autoEndK:]
        if debug:print(n)
        if n==0 or (lastK and lastK.count(n)==autoEndK):break
        conflict=randomConflict(current,nTeams)
        other=minSwap(current,conflict,nTeams)
        current[conflict],current[other]=current[other],current[conflict]
    return current,i#i!=maxSteps-1

In [17]:
def kMinConflictsSolutions(n,k):
    solutions=[]
    avrgTime,avrgSteps,avrgResets=0,0,0
    maxSteps=n*15
    while len(solutions)<k:
        start=time.time()
        sol,steps=minConflicts(n,maxSteps,debug=False)
        t=time.time()-start
        if nConflicts(sol,n)==0:
            solutions.append(tuple(sol))
            avrgTime+=t
            avrgSteps+=steps
        else:
            avrgResets+=1
    return solutions,list(set(solutions)),avrgTime/k,avrgSteps/k,avrgResets/k,maxSteps

In [482]:
for n in range(20,32,2):
    sols,unique,t,steps,resets,maxSteps=kMinConflictsSolutions(n,100)
    with open("minConflictsSols/"+str(n)+".json","w") as f:
        f.write(json.dumps(sols))
    print(n,len(unique),t,steps,resets,maxSteps)

KeyboardInterrupt: 

In [483]:
minConflicts(32,1000,debug=True)

88
87
85
83
82
78
76
75
73
71
70
68
66
64
63
63
63
62
60
60
60
59
57
56
55
55
54
54
53
53
53
53
52
51
51
51
49
49
49
47
47
46
45
45
45
45
45
43
43
43
43
43
40
40
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
37
36
36
36
34
34
34
34
33
33
33
33
30
30
30
30
28
28
28
28
28
28
28
28
28
28
28
28
28
28
26
26
26
26
24
24
24
24
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
21
20
20
18
18
18
18
16
16
16
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
12
12
12
12
12
12
10
10
10
10
10
10
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


([(30, 11),
  (32, 9),
  (3, 7),
  (13, 2),
  (20, 29),
  (8, 22),
  (19, 5),
  (10, 24),
  (27, 14),
  (4, 1),
  (21, 18),
  (25, 17),
  (23, 15),
  (6, 28),
  (16, 12),
  (31, 26),
  (30, 18),
  (24, 20),
  (7, 17),
  (3, 5),
  (27, 19),
  (11, 22),
  (8, 2),
  (32, 28),
  (23, 14),
  (9, 10),
  (29, 31),
  (26, 15),
  (4, 6),
  (16, 25),
  (12, 13),
  (1, 21),
  (20, 8),
  (11, 9),
  (17, 12),
  (13, 3),
  (24, 18),
  (31, 25),
  (19, 16),
  (5, 1),
  (32, 30),
  (4, 23),
  (21, 7),
  (2, 15),
  (26, 27),
  (28, 10),
  (22, 6),
  (14, 29),
  (31, 3),
  (10, 30),
  (22, 24),
  (14, 16),
  (29, 25),
  (26, 17),
  (11, 5),
  (6, 27),
  (9, 8),
  (7, 2),
  (13, 32),
  (21, 19),
  (18, 23),
  (12, 15),
  (28, 4),
  (20, 1),
  (30, 31),
  (2, 19),
  (4, 10),
  (25, 3),
  (16, 8),
  (15, 22),
  (11, 14),
  (18, 5),
  (28, 13),
  (20, 7),
  (23, 12),
  (1, 27),
  (6, 21),
  (29, 26),
  (24, 9),
  (32, 17),
  (17, 7),
  (14, 21),
  (13, 9),
  (3, 20),
  (32, 15),
  (29, 2),
  (12, 1),
  (16,

In [418]:
# Average nConflicts per nTeams of randomAssignment and 
n=6
s=0
for n in range(4,32,2):
    a=0
    s=0
    for i in range(100):
        s+=nConflicts(randomAssignment(n),n)
        a+=nConflicts(approxAssignment(n),n)
        
    print(n,s/100,a/100,n*(n-1),(s/100)/(n*(n-1)),(a/100)/(n*(n-1)))

4 9.76 0.0 12 0.8133333333333334 0.0
6 24.59 5.29 30 0.8196666666666667 0.17633333333333334
8 47.65 9.65 56 0.8508928571428571 0.17232142857142857
10 76.86 15.68 90 0.854 0.17422222222222222
12 112.26 22.46 132 0.8504545454545455 0.17015151515151516
14 155.02 31.0 182 0.8517582417582418 0.17032967032967034
16 205.75 36.44 240 0.8572916666666667 0.15183333333333332
18 263.11 42.8 306 0.8598366013071896 0.13986928104575164
20 327.54 51.93 380 0.8619473684210527 0.13665789473684212
22 397.01 58.68 462 0.8593290043290043 0.12701298701298702
24 473.98 68.1 552 0.8586594202898551 0.1233695652173913
26 557.79 76.77 650 0.8581384615384615 0.1181076923076923
28 650.39 85.33 756 0.8603042328042327 0.11287037037037037
30 746.78 94.95 870 0.858367816091954 0.10913793103448276


## Genetic algo

Tenemos que definir operadores de mutacion y de reproduccion. Lo que se me ocurrio es que el de mutacion puede ser un swap aleatorio y el de cruzamiento.

In [19]:
def mutate(assignment):
    # Random swap
    i,j=np.random.choice(len(assignment),size=2,replace=False)
    assignment=assignment[:]
    assignment[i],assignment[j]=assignment[j],assignment[i]
    return assignment

In [20]:
def freeMutate(assignment,nTeams):
    a=assignment[:]
    k=random.randint(0,len(a)-1)
    a[k]=random.choice(games(nTeams))
    return a

In [21]:
def crossover(a,b):
    assert len(a)==len(b),"a and b must be same length"
    k=random.randint(0,len(a)) # Random crossover point
    #print(k)
    offspring1=a[:k]
    for game in b:
        if len(offspring1)==len(a):break
        if game not in offspring1:
            offspring1.append(game)
    
    #print(offspring1,len(offspring1))
    assert len(offspring1)==len(a)
    return offspring1

In [22]:
def freeCrossover(a,b):
    assert len(a)==len(b),"a and b must be same length"
    k=random.randint(0,len(a)) # Random crossover point
    offspring1=a[:k]+b[k:]
    offspring2=b[:k]+a[k:]
    assert len(offspring1)==len(a) and len(offspring2)==len(a)
    return offspring1,offspring2

In [23]:
def hamming(a,b):
    dist=0
    assert len(a)==len(b)
    for i in range(len(a)):
        if a[i]!=b[i]:
            dist+=1
    return dist

In [24]:
def hammingOfList(l):
    dist=0
    for i in range(len(l)):
        for j in range(i+1,len(l)):
            dist+=hamming(l[i],l[j])
    return dist

In [25]:
a=randomAssignment(5)
b=randomAssignment(5)

In [121]:
# Given an assignment, returns the sum of the time between games of pairs of teams
def timeBetweenPairs(assignment,nTeams):
    pairs=len(assignment)/2
    time=0
    # For every pair of teams
    for i in range(1,nTeams+1):
        for j in range(i+1,nTeams+1):
            if i==j:continue
            if (i,j) in assignment and (j,i) in assignment:
                gameA=assignment.index((i,j))
                gameB=assignment.index((j,i))
                time+=abs(gameA-gameB)
                print(i,j,abs(gameA-gameB))
            else:
                pairs-=1
                print(i,j,-1)
      
    maxPossible=(pairs)**2
    out=time/maxPossible
    print(pairs,time,maxPossible,out) 
    assert out<1.1,(assignment,out)
    return out

In [122]:
timeBetweenPairs([(1, 4), (3, 2), (4, 3), (2, 1), (3, 1), (4, 2), (1, 3), (2, 4), (1, 2), (4, 1), (3, 1), (4, 2)],4)

1 2 5
1 3 2
1 4 9
2 3 -1
2 4 2
3 4 -1
4.0 18 16.0 1.125


AssertionError: ([(1, 4), (3, 2), (4, 3), (2, 1), (3, 1), (4, 2), (1, 3), (2, 4), (1, 2), (4, 1), (3, 1), (4, 2)], 1.125)

In [112]:
r=randomAssignment(4)
r

[(2, 4),
 (1, 4),
 (3, 4),
 (4, 1),
 (2, 1),
 (4, 3),
 (1, 3),
 (4, 2),
 (3, 2),
 (2, 3),
 (3, 1),
 (1, 2)]

In [99]:
r[0]=(1,3)

In [100]:
timeBetweenPairs(r,nTeams)

0.001767527985859776

In [27]:
def alterningScore(assignment, nTeams):
    teams={str(i):[] for i in range(1,nTeams+1)}
    
    for game in assignment:
        teams[str(game[0])].append(0) # 0 if local
        teams[str(game[1])].append(1) # 1 if visiting
        
    # Average of scores
    return np.array([aux_score(team) for team in teams.values()]).mean() 

def aux_score(arr):
    s=0
    previous=arr[0]
    for i in range(1,len(arr)):
        if arr[i]!=previous:
            s+=1
        previous=arr[i]
    return (s-1)/(len(arr)-2)

In [38]:
# Returns the proportion of weeks in which a city has a game
# given a tournament assignment and the teams in that city.
def cityOccupied(assignment,teamsInCity,nTeams):
    weeks=0
    nOccupied=0
    teamsInCity=set(teamsInCity)
    n=nTeams
    for i in range(0,len(assignment),n//2):
        weeks+=1
        week=assignment[i:i+n//2]
        occupied=False
        for game in week:
            if game[0] in teamsInCity:
                occupied=True
                break
        if occupied:
            nOccupied+=1
    return nOccupied/weeks          

In [48]:
# Returns the average of cityOccupied() for every city.
def citiesAlwaysOccupied(assignment,cityTeams,nTeams):
    return np.array([cityOccupied(assignment,teamsInCity,nTeams) for teamsInCity in cityTeams.values()]).mean()

In [40]:
r=randomAssignment(4)
r

[(4, 2),
 (1, 2),
 (2, 4),
 (2, 3),
 (2, 1),
 (1, 4),
 (4, 1),
 (4, 3),
 (3, 4),
 (1, 3),
 (3, 1),
 (3, 2)]

In [41]:
distances=randomCityDistances(3)
distances

array([[58. , 62. , 24. ],
       [62. , 34. , 46.5],
       [24. , 46.5, 58. ]])

In [42]:
cityTeams=randomCityTeams(4,3)
cityTeams

{1: [3], 2: [4], 3: [2, 1]}

In [43]:
teamCity=teamToCity(cityTeams)

In [44]:
distanceTraveled(r,1,teamCity,distances)

582.0

In [35]:
# Returns the sum of the distance travelled by each team in an assignment
def totalDistanceTravelled(assignment,cityTeams,cityDistances,nTeams):
    teamsCity=teamToCity(cityTeams)
    return sum([distanceTraveled(assignment,team,teamsCity,cityDistances) for team in range(1,nTeams+1)])  

In [36]:
# Assumes a Traveling Tournament Problem (i.e. teams don't return home)
def distanceTraveled(assignment,team,teamsToCity,cityDistances):
    distance=0
    city=teamsToCity[team]
    # Asume start at home
    for game in assignment:
        if team==game[1]: # If plays as visitor
            otherCity=teamsToCity[game[0]]
            distance+=2*cityDistances[city-1][otherCity-1]
            city=otherCity
        elif team==game[0]:
            otherCity=teamCity[team] # Home city
            distance+=2*cityDistances[city-1][otherCity-1]
            city=otherCity
    return distance

In [85]:
# Fitness function with which to eval individuals and determine their survaival.
# At first, we will only consider the number of conflicts.
# We can then add other soft requirements
def fitness(individual,nTeams,cityTeams,weights=[0.85,0.05,0.05,0.05]):
    assert abs(sum(weights)-1)<1e-5,sum(weights)
    # Normalizing nConflicts. To minimize.
    conflicts=1-(nConflicts(individual,nTeams)/len(individual))
    
    # timeBetweenPair of teams score. Normalized. To maximize.
    pairTimes=timeBetweenPairs(individual,nTeams)
    
    # alterningScore. Normalized. To maximize.
    alterning=alterningScore(individual,nTeams)
    
    # citiesAlwaysOccupied score. Normalized. To maximize.
    citiesAlways=citiesAlwaysOccupied(individual,cityTeams,nTeams)
    
    print(conflicts,pairTimes,alterning,citiesAlways)
    
    return (conflicts*weights[0]+pairTimes*weights[1]+alterning*weights[2]+citiesAlways*weights[3])

In [105]:
cityTeams=randomCityTeams(4,3)

In [115]:
# Genetic algo
nTeams=4
n=500 #size of population
nElite=int(n*0.05) #proportion best of population to retain
k=0.80 # can regulate "temperature"
mutationRate=0.2 # rate at which to mutate individuals
assert n%2==0,'N must be even to crossover'
# Random inicialitation of population
population=[approxAssignment(nTeams) for i in range(n)]
scores=[fitness(ind,nTeams,cityTeams) for ind in population]
for gen in range(5000): # Change with other stopping critiria (convergence, etc.)
    
    #min(0.5+(gen/1000),0.95)
    # SELECTION (Investigate other methods)
    # Elitist -> choose best n
    elite=sorted(population,key=lambda x:fitness(x,nTeams,cityTeams),reverse=True)[:nElite]
    
    # Tournament with remaining n-nElite
    fromTournament=[]
    for _ in range(n-nElite):
        a,b=np.random.choice(n,size=2,replace=False)
        best,worst=(a,b) if fitness(population[a],nTeams,cityTeams)>fitness(population[b],nTeams,cityTeams) else (b,a)
        if random.random()<k:
            fromTournament.append(population[best])
        else:
            fromTournament.append(population[worst])
    
    # CROSSOVER -> according to fitness
    #for i in range(0,len(population),2):
        #population.append(crossover(population[i],population[i+1]))
    #for i in range(0,len(population),2):
        #population.append(crossover(population[i],population[i+1]))
    #for i in range(len(population)//2):
        #population.append(crossover(population[i],population[len(population)-1-i]))
    
    population=[]
    for _ in range(n-nElite):
        a,b=np.random.choice(len(fromTournament),size=2,replace=False)
        population.append(freeCrossover(fromTournament[a],fromTournament[b])[0])
    
    """
    population=[] 
    scores=[fitness(ind,nTeams) for ind in fromTournament]
    temp=[1/-i if i<0 else 2 for i in scores]
    s=sum(temp)
    ps=[i/s for i in temp]
    #print(scores)
    for i in range(len(fromTournament)):
        a,b=np.random.choice(len(fromTournament),size=2,replace=False,p=ps)
        population.append(crossover(fromTournament[a],fromTournament[b])) 
    """
    """
    scores=[fitness(ind,nTeams) for ind in population]
    temp=[1/-i if i<0 else 2 for i in scores]
    s=sum(temp)
    ps=[i/s for i in temp]
    #print(scores)
    for i in range(int(len(population)**0.5)):
        a,b=np.random.choice(n,size=2,replace=False,p=ps)
        population.append(crossover(population[a],population[b]))
    """
     
    # MUTATION
    for i in range(len(population)):
        if random.random()<mutationRate:
            population[i]=freeMutate(population[i],nTeams)
    
    #Add elite
    population=elite+population
    assert len(population)==n
    
    # LOGGING
    scores=[fitness(ind,nTeams,cityTeams) for ind in population]
    conf=np.array([nConflicts(ind,nTeams) for ind in population])
    #print(','.join([str(int(i)) for i in scores]),np.array(scores).mean())
    print(gen,k,np.array(scores).mean(),np.array(scores).min(),np.array(scores).max(),conf.mean(),conf.min(),(np.array(scores)==0).sum())
    
    # STOPPING CRITIREA -> if 90% of population are valid sols. (Change later)
    if (np.array(scores)==0).sum()>=len(population)*0.9:
        break

print(gen,"generations")

1.0 1.0 0.375 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.25 0.5
1.0 0.5555555555555556 0.375 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.625 0.5
1.0 1.0 0.25 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.625 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 0.3333333333333333 0.75 0.5
1.0 1.0 0.125 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.625 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.375 0.5
1.0 0.3333333333333333 0.875 0.5
1.0 0.7777777777777778 0.75 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 1.0 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.625 0.5
1.0 1.0 0.375 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.375 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 0.3333333333333333 0.625 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.5 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.37

1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.25 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.3333333333333333 0.75 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.5 0.5
1.0 1.0 0.375 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.5 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.125 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 0.5555555555555556 0.875 0.5
1.0 1.0 0.25 0.5
1.0 1.0 0.25 0.5
1.0 1.0 0.5 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.5 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.5 0.5
1.0 0.5555555555555556 0.375 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.3333333333333333 0.75 0.5
1.0 1.0 0.25 0.5
1.0 1.0 0.5 0.5
1.0 0.5555555555555556 0.75 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.125 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.375 0.5
1.0 0.3333333333333333 0.875 0.5
1.0 0.7777777777777778 0.5 0.

1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.625 0.5
1.0 1.0 0.5 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.75 0.5
1.0 0.5555555555555556 0.375 0.5
1.0 1.0 0.125 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.25 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 1.0 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.25 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 0.7777777777777778 0.75 0.5
1.0 0.3333333333333333 0.75 0.5
1.0 0.3333333333333333 0.75 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.625 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.5 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.125 0.5
1.0 1.0 0.5 0.5
1.0 0.3333333333333333 0.875 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.5 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.625 0

1.0 0.7777777777777778 0.625 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.5 0.5
1.0 0.7777777777777778 0.75 0.5
1.0 1.0 0.5 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.75 0.5
1.0 1.0 0.25 0.5
1.0 1.0 0.375 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 1.0 0.625 0.5
1.0 0.7777777777777778 0.375 0.5
1.0 0.7777777777777778 0.75 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 1.0 0.375 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.75 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.5 0.5
1.0 1.0 0.125 0.5
1.0 0.5555555555555556 0.375 0.5
1.0 0.7777777777777778 0.625 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.5555555555555556 0.5 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.5 0.5
1.0 0.3333333333333333 0.875 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.375 0.5
1.0 0.7777777777777778 0.75 0.5
1.0 1.0 0.25 0.5
1.0 0.7777777777777778 0.5 0.5
1.0 1.0 0.625 0.5
1.0 0.5555555555555556 0.625 0.5
1.0 1.0 0.375 0.5
1.0 1.0 0.5 

AssertionError: ([(1, 4), (3, 2), (4, 3), (2, 1), (3, 1), (4, 2), (1, 3), (2, 4), (1, 2), (4, 1), (3, 1), (4, 2)], 1.125)

In [86]:
fitness(population[0],nTeams,cityTeams)

0.8416666666666667 7.329639889196676 0.4727458928724223 0.4119047619047619


1.1261311938653598

In [84]:
nConflicts(population[0],nTeams)

38

In [560]:
unique=set()
for i in population:
    if nConflicts(i,nTeams)==0:
        unique.add(tuple(i))

In [561]:
len(unique)

2

In [549]:
for i in unique:
    print(i)
    for team in teamsInGames(list(i)):
        if teamsInGames(list(i)).count(team)!=nTeams:
            print(team,teamsInGames(list(i)).count(team),len(i))
    print()

((11, 5), (2, 14), (15, 7), (4, 3), (8, 13), (12, 10), (1, 16), (6, 9), (5, 4), (7, 14), (3, 12), (1, 6), (16, 9), (13, 11), (2, 8), (10, 15), (2, 5), (3, 8), (7, 15), (16, 13), (11, 4), (10, 14), (12, 9), (6, 1), (11, 16), (5, 3), (9, 13), (6, 4), (8, 12), (10, 7), (15, 14), (2, 1), (9, 5), (16, 8), (7, 10), (1, 15), (6, 2), (13, 3), (11, 12), (14, 4), (2, 12), (5, 7), (4, 10), (9, 14), (3, 1), (16, 6), (15, 8), (11, 13), (12, 1), (6, 8), (2, 16), (4, 14), (10, 3), (15, 13), (7, 5), (11, 9), (7, 3), (8, 10), (15, 9), (5, 12), (13, 6), (16, 2), (11, 14), (4, 1), (8, 11), (4, 7), (3, 15), (10, 12), (13, 9), (14, 1), (5, 16), (2, 6), (3, 9), (4, 8), (13, 14), (2, 7), (1, 12), (15, 10), (5, 11), (6, 16), (16, 5), (10, 4), (12, 6), (8, 2), (9, 1), (15, 11), (7, 13), (3, 14), (7, 11), (3, 10), (5, 8), (6, 14), (12, 16), (9, 2), (13, 4), (15, 1), (12, 5), (16, 10), (14, 13), (4, 15), (11, 3), (8, 1), (6, 7), (2, 9), (6, 10), (14, 16), (12, 4), (11, 7), (1, 2), (8, 9), (15, 3), (5, 13), (1, 1